In [ ]:
import os
import math
import numpy as np

import torch
from torchvision import transforms
from torchvision.transforms import functional as TF
from torch.utils.data import DataLoader
from datasets import load_dataset
from tqdm import tqdm

from GeospatialFM.datasets.ssl4eo.utils import get_ssl4eo_metadata
from GeospatialFM.data_process import apply_normalization, pretrain_transform, multimodal_collate_fn
from GeospatialFM.models import SpatialSpectralLowRankViTConfig, SpatialSpectralMAEViT
from GeospatialFM.models.low_rank_attention import get_perception_field_mask
from GeospatialFM.models import PositionalChannelEmbedding
from GeospatialFM.datasets.ssl4eo import SSL4EODataset
from GeospatialFM.scripts.trainer import MAETrainer

from functools import partial
from accelerate import Accelerator
from transformers import TrainingArguments

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import rasterio
import pandas as pd
import json

%load_ext autoreload
%autoreload 2

## Perception Field Mask

In [ ]:
patch_size = 16
num_patches = 64
perception_field_mask = get_perception_field_mask(num_patches, patch_size, 10, attention_radius=640, cls_token=False)
perception_field_mask.shape
plt.imshow(perception_field_mask.numpy())
# plt.axis('off')
plt.xticks([-0.5, num_patches-0.5], labels=[0, num_patches])
plt.yticks([-0.5, num_patches-0.5], labels=[0, num_patches])
# plt.title("Attention Mask Example: 196x196", fontsize=10, fontweight='bold')
# plt.show()
plt.savefig("perception_field_attention_mask.pdf", bbox_inches='tight', pad_inches=0.)

In [ ]:
one_perception = perception_field_mask[0].reshape(int(math.sqrt(perception_field_mask.shape[1])), int(math.sqrt(perception_field_mask.shape[1]))).numpy()
idx = torch.rand(num_patches)
keep_idx = torch.argsort(idx)[:int(num_patches*0.25)]
random_mask = torch.zeros(num_patches)
random_mask[keep_idx] = 1
random_mask = random_mask.reshape(int(math.sqrt(num_patches)), int(math.sqrt(num_patches)))
fig, ax = plt.subplots(1, 3, figsize=(14, 4))
ax[0].imshow(random_mask.numpy())
ax[0].set_title("Random MAE Mask")
ax[1].imshow(one_perception, vmin=0, vmax=1)
ax[1].set_title("One Perception Field Mask")
ax[2].imshow(random_mask.numpy() * one_perception)
ax[2].set_title("Actual MAE Mask")
plt.show()
# print(perception_field_mask[0].reshape(int(math.sqrt(perception_field_mask.shape[1])), int(math.sqrt(perception_field_mask.shape[1]))).numpy().sum())

In [ ]:
# apply random mask to perception field mask
random_mask_idx = random_mask.reshape(-1).nonzero().squeeze()
perception_field_mask_masked = perception_field_mask[random_mask_idx][:, random_mask_idx]
plt.imshow(perception_field_mask_masked.numpy())

## PositionalChannel Embedding

In [ ]:
pos_chan_embed = PositionalChannelEmbedding(embed_dim=768)
tokens = torch.randn(1, 5, 64, 768)
# channel_ids = torch.arange(5).unsqueeze(0) * 1000
channel_ids = torch.tensor([492.4, 559.8, 664.6, 1613.7, 2202.4]).unsqueeze(0)
pos_embed = pos_chan_embed(tokens, 10, channel_ids, cls_token=False).squeeze(0)
print(pos_embed.shape)

In [ ]:
# print the embedding of the first channel
plt.figure(figsize=(10, 10))
plt.imshow(torch.cat([pos_embed[i] for i in range(5)], dim=0), cmap="inferno")
# add tick label to y axis
plt.yticks(np.arange(11)*32-0.5, labels=[f"$\lambda$ = {float(channel_ids[0, i//2]):.1f} $\mu m$" if i % 2 == 1 else "" for i in range(11)], fontsize=12, fontweight='bold')
# 
plt.xticks(np.array([0, 96, 192, 288, 384, 480, 576, 672, 768])-0.5, labels=[0, 96, 192, 288, 384, 480, 576, 672, 768], fontsize=12)
# plt.axis('off')
plt.xlabel("Embedding Dimension", fontsize=12, fontweight='bold')
plt.tight_layout()
plt.savefig("positional_channel_embedding.pdf", bbox_inches='tight', pad_inches=0)

## SSL4EO

In [2]:
metadata = get_ssl4eo_metadata()
optical_mean, optical_std = metadata["s2c"]["mean"], metadata["s2c"]["std"]
radar_mean, radar_std = metadata["s1"]["mean"], metadata["s1"]["std"]

In [3]:
# dataset = load_dataset("GeospatialFM/datasets/ssl4eo", data_dir="/home/haozhesi/Dropbox/GeospatialFM/data/geospatial/SSL4EO", keep_in_memory=False)
dataset = dict(train=SSL4EODataset(root="/home/haozhesi/Dropbox/GeospatialFM/data/geospatial/SSL4EO"))
apply_transform = partial(apply_normalization, optical_mean=optical_mean, optical_std=optical_std, radar_mean=radar_mean, radar_std=radar_std)
# dataset = dataset.map(apply_transform)

In [ ]:
def norm_img(img):
    return (img - img.min()) / (img.max() - img.min())

data_idx = np.random.randint(0, len(dataset['train']))
data_idx = 117967
# print(data_idx)
sample = apply_transform(dataset['train'][data_idx])
sample_optical_image = torch.tensor(sample['optical']).numpy()
sample_radar_image = torch.tensor(sample['radar']).numpy()
sample_image = np.concatenate([sample_optical_image, sample_radar_image], axis=0)
bands = metadata["s2c"]["bands"] + metadata["s1"]["bands"]
rgb_image = sample_optical_image[[3, 2, 1]].transpose(1, 2, 0)
rgb_image = norm_img(rgb_image)

# visualize a data sample
# mpl.rcParams['axes.titlesize'] = 28
# mpl.rcParams['axes.labelsize'] = 24
# mpl.rcParams['xtick.labelsize'] = 18
# mpl.rcParams['ytick.labelsize'] = 18

# Number of images excluding the RGB image
num_images = sample_image.shape[0]
# pick the 75 quantile location of each channel
random_locs = []
target_labels = []

for i in [0, 0.25, 0.5, 0.75, 1]:
    quantile = int(sample_image[1].size*i) if i != 1 else -1
    loc = np.unravel_index(np.argsort(sample_image[1].flatten())[quantile], sample_image[1].shape)
    random_locs.append(loc)
    target_labels.append(f"{i*100}%")

# Create a figure
fig = plt.figure(figsize=(32, 9))  # Adjust the figure size as needed

# Create a GridSpec with 3 rows and 8 columns
gs = GridSpec(3, 11, figure=fig)

ax_img = fig.add_subplot(gs[:, :3])
ax_img.imshow(rgb_image)
ax_img.axis('off')
ax_img.set_title('RGB Image')
for loc in random_locs:
    ax_img.plot(loc[1], loc[0], 'x', markersize=10, markeredgewidth=4)

# The rest of the images (adjust the range according to your number of images)
for i in range(num_images):
    # Calculate the position for the current image
    row = i // 5
    col = i % 5 
    ax = fig.add_subplot(gs[row, col+3])
    ax.imshow(sample_image[i])  # Replace with each individual image array
    # plot cross at random locations
    for loc in random_locs:
        ax.plot(loc[1], loc[0], 'x', markersize=10, markeredgewidth=4)
    # bold the title
    ax.set_title(f"{bands[i]}")  # Replace with the title for each image
    ax.axis('off')

ax_curve = fig.add_subplot(gs[:, -3:])
for loc, label in zip(random_locs, target_labels):
    ax_curve.plot(np.arange(len(bands)), sample_image[:, loc[0], loc[1]], linewidth=3, marker='o', markersize=6, label=label)
# ax_curve.legend(loc='upper right', fontsize=18)
# ax_curve.set_xlabel('Bands')
ax_curve.set_ylabel('Normalized Reflectance')
ax_curve.set_title('Spectral Signature at Marked Locations')
ax_curve.set_yticks([])
# set the band names as xticks
ax_curve.set_xticks(np.arange(len(bands)))
ax_curve.set_xticklabels(bands, rotation=90)

plt.tight_layout()
plt.show()
# save as pdf
# plt.savefig('spectral_signature.pdf', bbox_inches='tight', pad_inches=0.1)

In [ ]:
# os.makedirs("patches/masked")
# idx = torch.rand(16)
# keep_idx = torch.argsort(idx)[:int(16*0.5)]
# random_mask = torch.zeros(16)
# random_mask[keep_idx] = 1

# print(random_mask)
# channel_idx = torch.rand(num_images)
# keep_channel_idx = torch.argsort(channel_idx)[:int(num_images*0.5)]
# randon_channel_mask = torch.zeros(num_images) 
# randon_channel_mask[keep_channel_idx] = 1

# cut the sample image into 3x3 patches
patch_size = 264//3
channel_indices = [1, 2, 3, 4, 13, 14]

fig, ax = plt.subplots(7, 7, figsize=(10, 10))
for i in range(7):
    for j in range(7):
        if i == 0 and j == 0:
            # fill in yellowish red pure color
            ax[j, i].imshow(np.ones((patch_size, patch_size))*0.8, cmap='Oranges', vmin=0, vmax=1)
        elif i == 0:
            ax[j, i].imshow(np.ones((patch_size, patch_size))*0.2, cmap='YlOrRd', vmin=0, vmax=1)
        elif j == 0:
            ax[j, i].imshow(np.ones((patch_size, patch_size))*0.3, cmap='Greens', vmin=0, vmax=1)
        else:
            plot_i = i-1
            plot_j = j-1
            ax[j, i].imshow(sample_image[channel_indices[plot_i], plot_j//3*patch_size:(plot_j//3+1)*patch_size, plot_j%3*patch_size:(plot_j%3+1)*patch_size])
        ax[j, i].axis('off')
plt.tight_layout()
# plt.show()
plt.savefig("sample_patches.pdf", bbox_inches='tight', pad_inches=0)

In [ ]:
patch_size = 264//3
channel_indices = [1, 2, 3, 4, 13, 14]
masked_channel_indices = [1, 3, 4]
masked_positions = [0, 1, 3, 5]
print(masked_channel_indices, masked_positions)

fig, ax = plt.subplots(7, 7, figsize=(10, 10))
for i in range(7):
    for j in range(7):
        if i == 0 and j == 0:
            ax[j, i].imshow(np.ones((patch_size, patch_size))*0.8, cmap='Oranges', vmin=0, vmax=1)
        elif i == 0:
            if j-1 in masked_positions:
                ax[j, i].imshow(np.ones((patch_size, patch_size))*0.4, cmap='Greys', vmin=0, vmax=1)
            else:
                ax[j, i].imshow(np.ones((patch_size, patch_size))*0.2, cmap='YlOrRd', vmin=0, vmax=1)
        elif j == 0:
            if i-1 in masked_channel_indices:
                ax[j, i].imshow(np.ones((patch_size, patch_size))*0.4, cmap='Greys', vmin=0, vmax=1)
            else:
                ax[j, i].imshow(np.ones((patch_size, patch_size))*0.3, cmap='Greens', vmin=0, vmax=1)
        else:
            plot_i = i-1
            plot_j = j-1
            if plot_i in masked_channel_indices or plot_j in masked_positions:
                ax[j, i].imshow(np.ones((patch_size, patch_size))*0.4, cmap='Greys', vmin=0, vmax=1)
            else:
                ax[j, i].imshow(sample_image[channel_indices[plot_i], plot_j//3*patch_size:(plot_j//3+1)*patch_size, plot_j%3*patch_size:(plot_j%3+1)*patch_size])
        ax[j, i].axis('off')
plt.tight_layout()
# plt.show()
plt.savefig("masked_patches.pdf", bbox_inches='tight', pad_inches=0)

In [ ]:
patch_size = 264//3
channel_indices = [1, 2, 3, 4, 13, 14]
masked_channel_indices = [1, 3, 4]
unmasked_channel_indices = [0, 2, 5]
masked_positions = [0, 1, 3, 5]
unmasked_positions = [2, 4]
print(masked_channel_indices, masked_positions)

fig, ax = plt.subplots(7, 7, figsize=(10, 10))
for i in range(7):
    for j in range(7):
        if i == 0 and j == 0:
            ax[j, i].imshow(np.ones((patch_size, patch_size))*0.8, cmap='Oranges', vmin=0, vmax=1)
        elif i == 0:
            ax[j, i].imshow(np.ones((patch_size, patch_size))*0.2, cmap='YlOrRd', vmin=0, vmax=1)
        elif j == 0:
            ax[j, i].imshow(np.ones((patch_size, patch_size))*0.3, cmap='Greens', vmin=0, vmax=1)
        else:
            try:
                plot_i = unmasked_channel_indices[i-1]
                plot_j = unmasked_positions[j-1]
                ax[j, i].imshow(sample_image[channel_indices[plot_i], plot_j//3*patch_size:(plot_j//3+1)*patch_size, plot_j%3*patch_size:(plot_j%3+1)*patch_size])
            except:
                ax[j, i].imshow(np.ones((patch_size, patch_size))*0.5, cmap='Greys', vmin=0, vmax=1)
        ax[j, i].axis('off')
plt.tight_layout()
# plt.show()
plt.savefig("masked_patches_2.pdf", bbox_inches='tight', pad_inches=0)


In [ ]:
def get_mask_and_visualization(image, image_size, patch_size=16, attention_radius=640, median_mask=0.55, alpha=0.3):
    num_patches = (image_size // patch_size)**2
    perception_field_mask = get_perception_field_mask(num_patches, patch_size, 10, attention_radius=attention_radius, cls_token=False)
    vis_perception_mask = perception_field_mask[int(num_patches*median_mask)].reshape(int(math.sqrt(perception_field_mask.shape[1])), int(math.sqrt(perception_field_mask.shape[1]))).numpy()
    vis_perception_mask = TF.resize(torch.tensor(vis_perception_mask).unsqueeze(0), size=(image_size, image_size), interpolation=TF.InterpolationMode.NEAREST).squeeze(0)
    
    image = image.copy()
    image = TF.center_crop(torch.tensor(image).permute(2, 0, 1), output_size=(image_size, image_size)).permute(1, 2, 0).numpy()
    applied_mask = (1-vis_perception_mask.unsqueeze(-1).numpy()) * (1-alpha) + (1-vis_perception_mask.unsqueeze(-1).numpy()) * image * alpha + (vis_perception_mask.unsqueeze(-1)).numpy()*image
    target_patch_idx = int(median_mask*num_patches)
    target_i = target_patch_idx // int(num_patches**0.5)
    target_j = target_patch_idx % int(num_patches**0.5)
    return perception_field_mask, applied_mask, target_i, target_j

vis_perception_mask_256, applied_mask_256, target_i_256, target_j_256 = get_mask_and_visualization(rgb_image, 256)
vis_perception_mask_224, applied_mask_224, target_i_224, target_j_224 = get_mask_and_visualization(rgb_image, 224)
vis_perception_mask_192, applied_mask_192, target_i_192, target_j_192 = get_mask_and_visualization(rgb_image, 192)
vis_perception_mask_128, applied_mask_128, target_i_128, target_j_128 = get_mask_and_visualization(rgb_image, 128)

fig, ax = plt.subplots(2, 2, figsize=(10, 10))  # Adjusted to 2x4 layout

ax[0, 0].imshow(applied_mask_256)
ax[0, 0].add_patch(plt.Rectangle((target_j_256*16, target_i_256*16), 16, 16, edgecolor='red', facecolor='none'))
ax[0, 0].set_title("Image Size: 256x256", fontsize=26, fontweight='bold')
ax[0, 0].axis('off')


ax[0, 1].imshow(applied_mask_224)
ax[0, 1].set_title("Image Size: 224x224", fontsize=26, fontweight='bold')
ax[0, 1].add_patch(plt.Rectangle((target_j_224*16, target_i_224*16), 16, 16, edgecolor='red', facecolor='none'))
ax[0, 1].axis('off')

ax[1, 0].imshow(applied_mask_192)
ax[1, 0].set_title("Image Size: 192x192", fontsize=26, fontweight='bold')
ax[1, 0].add_patch(plt.Rectangle((target_j_192*16, target_i_192*16), 16, 16, edgecolor='red', facecolor='none'))
ax[1, 0].axis('off')

ax[1, 1].imshow(applied_mask_128)
ax[1, 1].set_title("Image Size: 128x128", fontsize=28, fontweight='bold')
ax[1, 1].add_patch(plt.Rectangle((target_j_128*16, target_i_128*16), 16, 16, edgecolor='red', facecolor='none'))
ax[1, 1].axis('off')
plt.tight_layout()
plt.show()
# title
# plt.suptitle("Perception Field Attention Mask", fontsize=18, fontweight='bold')
# plt.savefig("perception_field_mask.pdf", bbox_inches='tight', pad_inches=0.)

In [ ]:
fig, ax = plt.subplots(2, 4, figsize=(20, 10))  # Adjusted to 2x4 layout

ax[0, 0].imshow(vis_perception_mask_256)
ax[0, 0].set_title("Patch Number: 256", fontsize=26, fontweight='bold')
ax[0, 0].axis('off')

ax[0, 1].imshow(applied_mask_256)
ax[0, 1].add_patch(plt.Rectangle((target_j_256*16, target_i_256*16), 16, 16, edgecolor='red', facecolor='none'))
ax[0, 1].set_title("Image Size: 256x256", fontsize=26, fontweight='bold')
ax[0, 1].axis('off')

ax[0, 2].imshow(vis_perception_mask_224)
ax[0, 2].set_title("Patch Number: 196", fontsize=26, fontweight='bold')
ax[0, 2].axis('off')

ax[0, 3].imshow(applied_mask_224)
ax[0, 3].set_title("Image Size: 224x224", fontsize=26, fontweight='bold')
ax[0, 3].add_patch(plt.Rectangle((target_j_224*16, target_i_224*16), 16, 16, edgecolor='red', facecolor='none'))
ax[0, 3].axis('off')

ax[1, 0].imshow(vis_perception_mask_192)
ax[1, 0].set_title("Patch Number: 144", fontsize=26, fontweight='bold')
ax[1, 0].axis('off')

ax[1, 1].imshow(applied_mask_192)
ax[1, 1].set_title("Image Size: 192x192", fontsize=26, fontweight='bold')
ax[1, 1].add_patch(plt.Rectangle((target_j_192*16, target_i_192*16), 16, 16, edgecolor='red', facecolor='none'))
ax[1, 1].axis('off')

ax[1, 2].imshow(vis_perception_mask_128)
ax[1, 2].set_title("Patch Number: 64", fontsize=26, fontweight='bold')
ax[1, 2].axis('off')

ax[1, 3].imshow(applied_mask_128)
ax[1, 3].set_title("Image Size: 128x128", fontsize=26, fontweight='bold')
ax[1, 3].add_patch(plt.Rectangle((target_j_128*16, target_i_128*16), 16, 16, edgecolor='red', facecolor='none'))
ax[1, 3].axis('off')
plt.tight_layout()
plt.savefig("perception_field_mask_visualization.pdf", bbox_inches='tight', pad_inches=0)

In [ ]:
# plt.imshow(((1-vis_perception_mask_256.unsqueeze(-1).numpy()) * img_256 * 0.3))
plt.axis('off')
plt.show()

## Dataloader with Image Size Changing

In [11]:
batch_size = 8

collate_fn = partial(multimodal_collate_fn, transform=pretrain_transform, normalization=apply_transform)

train_loader = DataLoader(
    dataset['train'], 
    batch_size=batch_size, 
    shuffle=True, 
    num_workers=4, 
    collate_fn=collate_fn
)

# for n, batch in enumerate(train_loader):
#     print(batch['optical'].shape)
#     if n == 4:
#         break

## Model

In [ ]:
# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Define configuration
config = SpatialSpectralLowRankViTConfig(
    patch_size=16,
    embed_dim=768,
    depth=12,
    num_heads=12,
    decoder_embed_dim=512,
    decoder_depth=8,
    decoder_num_heads=16,
    use_perception_field_mask=True,
    attention_radius=640,
    norm_pix_loss=False,
)

# Initialize model
model = SpatialSpectralMAEViT(config)

model

In [ ]:
# find the number of parameters
num_params = sum(p.numel() for p in model.encoder.parameters())
print(f"Number of parameters: {num_params}")

## Training Code

In [8]:
# Set model to training mode
model.to('cuda:0', dtype=torch.bfloat16)
model.train()

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

# # load checkpoint
# model.load_state_dict(torch.load("/home/haozhesi/Dropbox/GeospatialFM/results/models/LRSSVIT/checkpoint-1000/pytorch_model.bin"))

In [ ]:
for batch in tqdm(train_loader):
    optical = batch['optical'].to('cuda:0', dtype=torch.bfloat16)
    radar = batch['radar'].to('cuda:0', dtype=torch.bfloat16)
    optical_channel_wv = batch['optical_channel_wv']
    radar_channel_wv = batch['radar_channel_wv']
    spatial_resolution = batch['spatial_resolution']
    # Run forward pass
    output = model(
        optical=optical,
        radar=radar,
        optical_channel_wv=optical_channel_wv,
        radar_channel_wv=radar_channel_wv,
        mask_ratio=0.75,
        channel_mask_ratio=0.5,
        spatial_resolution=spatial_resolution,
    )
    break

# Check output
expected_keys = [
    'target',
    'optical_channel_mask', 'optical_recon', 'optical_pos_mask',
    'radar_channel_mask', 'radar_recon', 'radar_pos_mask',
    'multi_channel_mask', 'multi_recon', 'multi_pos_mask'
]

for key in expected_keys:
    assert key in output, f"Missing key in output: {key}"
    assert isinstance(output[key], torch.Tensor), f"Output {key} is not a tensor"

In [ ]:
# manually compute the loss
loss = {}
total_loss = 0
target = output['target']
for modal in ['optical', 'radar', 'multi']:
    if f'{modal}_recon' in output:
        recon = output[f'{modal}_recon']
        channel_mask = output[f'{modal}_channel_mask']
        pos_mask = output[f'{modal}_pos_mask']
        # positional MSE
        pos_loss = (torch.mean((recon - target) ** 2, dim=[1, 3]) * pos_mask).sum() / pos_mask.sum()
        # channel MSE
        channel_loss = (torch.mean((recon - target) ** 2, dim=[2, 3]) * channel_mask).sum() / channel_mask.sum()
        loss[f"{modal}_pos_loss"] = pos_loss
        loss[f"{modal}_channel_loss"] = channel_loss
        loss[f"{modal}_loss"] = pos_loss + channel_loss
        total_loss += loss[f"{modal}_loss"]
loss['total_loss'] = total_loss
print(loss)

In [11]:
# step
optimizer.zero_grad()
total_loss.backward()
optimizer.step()